In [14]:
!./vina --receptor ../../pdbbindR/10gs/new.pdbqt --ligand ../../pdbbindR/10gs/out1.pdbqt --score_only --log score.txt

Detected 8 CPUs
Affinity  -8.46751
Term1 30.17549
Term2 -101.04414
Term3 1.44796
Term4 0.27583
Term5 96.16501
Term6 142.47257
Term7 173.98751
Term8 208.47633
Term9 250.96594
Term10 302.71546
Term11 151.77830
Term12 291.90864
Term13 421.87711
Term14 207.40922
Term15 408.28327
Term16 594.43972
Term17 266.00411
Term18 523.52074
Term19 769.61977
Term20 1038.04925
Term21 471.55674
Term22 869.89245
Term23 1270.14648
Term24 1496.32030
Term25 1211.70835
Term26 682.72425
Term27 1154.53252
Term28 1581.90515
Term29 1732.46116
Term30 1432.56011
Term31 1134.83964
Term32 1597.19291
Term33 1932.20909
Term34 1998.79057
Term35 1753.71703
Term36 28.53358
Term37 12.45648
Term38 4.44719
Term39 1.33972
Term40 0.27984
Term41 0.02177
Term42 0.00000
Term43 0.00000
Term44 48.51258
Term45 65.53120
Term46 83.28231
Term47 119.45820
Term48 122.85338
Term49 -707.59756
Term50 3.84718
Term51 4.39699
Term52 5.54719
Term53  13.50000
Term54  11.50000
Term55  33.00000
Term56  14.00000
Term57  9.00000
Term58  10.00000


In [15]:
import pandas as pd
import numpy as np
from ecif import *
from joblib import delayed, Parallel

In [16]:

Protein = {'pdb': ('../../pdbbindR/10gs/new.pdbqt')}
Ligand = {'pdb': ('../../pdbbindR/10gs/out.sdf')}

list = GetECIF(Protein['pdb'], Ligand['pdb'], distance_cutoff=6.0)

In [17]:
features = {'pdb': list}
features_new = pd.DataFrame.from_dict(features).T
features_new.columns = PossibleECIF
features_new

,C;4;1;3;0;0-Br;1;1;0;0;0,C;4;1;3;0;0-C;3;3;0;1;1,C;4;1;3;0;0-C;4;1;1;0;0,C;4;1;3;0;0-C;4;1;2;0;0,C;4;1;3;0;0-C;4;1;3;0;0,C;4;1;3;0;0-C;4;2;0;0;0,C;4;1;3;0;0-C;4;2;1;0;0,C;4;1;3;0;0-C;4;2;1;0;1,C;4;1;3;0;0-C;4;2;1;1;1,C;4;1;3;0;0-C;4;2;2;0;0,...,S;2;2;0;0;0-S;2;1;1;0;0,S;2;2;0;0;0-S;2;2;0;0;0,S;2;2;0;0;0-S;2;2;0;0;1,S;2;2;0;0;0-S;2;2;0;1;1,S;2;2;0;0;0-S;3;3;0;0;0,S;2;2;0;0;0-S;3;3;0;0;1,S;2;2;0;0;0-S;4;3;0;0;0,S;2;2;0;0;0-S;6;4;0;0;0,S;2;2;0;0;0-S;6;4;0;0;1,S;2;2;0;0;0-S;7;4;0;0;0
pdb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
file = pd.read_csv('score.txt', index_col=False)
file.columns = ['feature value']
#file = file.tail(-1)
file = file['feature value'].str.split(r'\s', n=1, expand=True)
file.columns = ['feature', 'value']
term = file['value']
term = pd.to_numeric(term)
terms = term.to_list()
term = file['feature']
term = term.to_list()

In [19]:
def get_vina_terms(file):
    file = pd.read_csv(file, index_col=False)
    file.columns = ['feature value']
    #file = file.tail(-1)
    file = file['feature value'].str.split(r'\s', n=1, expand=True)
    file.columns = ['feature', 'value']
    term = file['value']
    term = pd.to_numeric(term)
    terms = term.to_list()
    return terms

In [20]:
refinedpdb = ['pdb']

verbose = 10
n_jobs = 2

Protein = {pdb: (f'score.txt') for pdb in refinedpdb}

with Parallel(n_jobs=n_jobs, verbose=verbose) as parallel:
    list = parallel(delayed(get_vina_terms)(Protein[pdb]) for pdb in refinedpdb)

features = {pdb: list for pdb, list in zip(refinedpdb, list)}
features_new1 = pd.DataFrame.from_dict(features).T
features_new1.columns = term

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Batch computation too fast (0.0159s.) Setting batch_size=2.
[Parallel(n_jobs=2)]: Done   1 out of   1 | elapsed:    0.0s finished


In [21]:
features_new

,C;4;1;3;0;0-Br;1;1;0;0;0,C;4;1;3;0;0-C;3;3;0;1;1,C;4;1;3;0;0-C;4;1;1;0;0,C;4;1;3;0;0-C;4;1;2;0;0,C;4;1;3;0;0-C;4;1;3;0;0,C;4;1;3;0;0-C;4;2;0;0;0,C;4;1;3;0;0-C;4;2;1;0;0,C;4;1;3;0;0-C;4;2;1;0;1,C;4;1;3;0;0-C;4;2;1;1;1,C;4;1;3;0;0-C;4;2;2;0;0,...,S;2;2;0;0;0-S;2;1;1;0;0,S;2;2;0;0;0-S;2;2;0;0;0,S;2;2;0;0;0-S;2;2;0;0;1,S;2;2;0;0;0-S;2;2;0;1;1,S;2;2;0;0;0-S;3;3;0;0;0,S;2;2;0;0;0-S;3;3;0;0;1,S;2;2;0;0;0-S;4;3;0;0;0,S;2;2;0;0;0-S;6;4;0;0;0,S;2;2;0;0;0-S;6;4;0;0;1,S;2;2;0;0;0-S;7;4;0;0;0
pdb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
core1_cry = pd.merge(features_new1, features_new, left_index=True, right_index=True)
core1_cry

,Affinity,Term1,Term2,Term3,Term4,Term5,Term6,Term7,Term8,Term9,...,S;2;2;0;0;0-S;2;1;1;0;0,S;2;2;0;0;0-S;2;2;0;0;0,S;2;2;0;0;0-S;2;2;0;0;1,S;2;2;0;0;0-S;2;2;0;1;1,S;2;2;0;0;0-S;3;3;0;0;0,S;2;2;0;0;0-S;3;3;0;0;1,S;2;2;0;0;0-S;4;3;0;0;0,S;2;2;0;0;0-S;6;4;0;0;0,S;2;2;0;0;0-S;6;4;0;0;1,S;2;2;0;0;0-S;7;4;0;0;0
pdb,-8.46751,30.17549,-101.04414,1.44796,0.27583,96.16501,142.47257,173.98751,208.47633,250.96594,...,0,0,0,0,0,0,0,0,0,0


In [27]:
import joblib

rf_model = joblib.load('../ML_SF_PDBBIND2020/MLSF_models/GBT_model_ecif_vina_redock_updated.sav')
y_pred_rf = rf_model.predict(core1_cry)

In [28]:
y_pred_rf

array([6.55530671])

In [ ]:
term